In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:
from transformers import AutoTokenizer
import haystack
import torch
import importlib

haystack = importlib.reload(haystack)

/Users/power/code/haystack/.conda/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


INFO:AnglE:Prompt is set, the prompt will be automatically applied during the encoding phase. To disable prompt setting, please configure set_prompt(prompt=None)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

model = haystack.HaystackForCausalLM.from_pretrained("google/gemma-2b")
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of HaystackForCausalLM were not initialized from the model checkpoint at google/gemma-2b and are newly initialized: ['model.db.db.weight', 'model.db.embedding_to_hidden.bias', 'model.db.embedding_to_hidden.weight', 'model.db.hidden_to_key.bias', 'model.db.hidden_to_key.weight', 'model.db.keys.bias', 'model.db.keys.weight', 'model.layers.0.db_layer.db.weight', 'model.layers.0.db_layer.embedding_to_hidden.bias', 'model.layers.0.db_layer.embedding_to_hidden.weight', 'model.layers.0.db_layer.hidden_to_key.bias', 'model.layers.0.db_layer.hidden_to_key.weight', 'model.layers.0.db_layer.keys.bias', 'model.layers.0.db_layer.keys.weight', 'model.layers.1.db_layer.db.weight', 'model.layers.1.db_layer.embedding_to_hidden.bias', 'model.layers.1.db_layer.embedding_to_hidden.weight', 'model.layers.1.db_layer.hidden_to_key.bias', 'model.layers.1.db_layer.hidden_to_key.weight', 'model.layers.1.db_layer.keys.bias', 'model.layers.1.db_layer.keys.weight', 'model.layers.10.db_layer.db.weight'

In [4]:
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/Users/power/code/haystack/.conda/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Position ids? None
Prepare:  torch.Size([1, 9]) torch.Size([1, 10]) tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
Layer:  0 torch.Size([1, 9, 2048]) torch.Size([1, 1, 8192, 8192]) torch.Size([1, 10]) DynamicCache() False True tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Before Rotary:  torch.Size([1, 8, 10, 256]) torch.Size([1, 1, 10, 256]) torch.Size([1, 10, 256]) torch.Size([1, 10, 256]) torch.Size([1, 10])
Rotary Pos Embed torch.Size([1, 8, 10, 256]) torch.Size([1, 1, 10, 256]) torch.Size([1, 1, 10, 256])
Layer:  1 torch.Size([1, 9, 2048]) torch.Size([1, 1, 8192, 8192]) torch.Size([1, 10]) DynamicCache() False True tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Before Rotary:  torch.Size([1, 8, 10, 256]) torch.Size([1, 1, 10, 256]) torch.Size([1, 10, 256]) torch.Size([1, 10, 256]) torch.Size([1, 10])
Rotary Pos Embed torch.Size([1, 8, 10, 256]) torch.Size([1, 1, 10, 256]) torch.Size([1, 1, 10, 256])
Layer:  2 torch.Size([1, 9, 2048]) torch.Size([1, 1, 8192, 8192]) torch.Size([1, 10]) DynamicCache() Fa

In [13]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
datasets

Retrying in 1s [Retry 1/5].


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets["train"][1]


{'input_ids': [2, 589, 164672, 4016, 81321, 5638, 589, 235248, 108],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [21]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [23]:
tokenizer.decode(lm_datasets["train"][10]["input_ids"])

'lian Army exploit the concept of plausible deniability in order to send them on missions that would otherwise make Gallia lose face in the war . While at times this works to their advantage , such as a successful incursion into Imperial territory , other orders cause certain members of the 422nd great distress . One such member , Gusurg , becomes so enraged that he abandons his post and defects into the ranks of Calamity Raven , attached to the ideal of Darcsen independence proposed by their leader , Dahau . At the same time , elements within Gallian Army Command move to erase the Nameless in order to protect their own'

In [25]:
from transformers import Trainer, TrainingArguments

model_name = 'haystack-v0'
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [26]:
trainer.train()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: